# Search Engine With Tools and Agents

In [1]:
!pip install wikipedia


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11704 sha256=45f8071243b23d3804a14b94398c73339ee91b20f11010b88193a4ac232d86cb
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [2]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=21438c39a4c934272efe10b2c199d8ba19aa2c846ae74059a3b99e778eb38652
  Stored in directory: c:\users\kumar\appdata\local\pip\cache\wheels\03\f5\1a\23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [3]:
## Importing tools and API Wrapper 
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import  ArxivAPIWrapper,WikipediaAPIWrapper